### QuickStart: Compare runs, Choose a model, and deploy it to the REST API ###

In this quickstart, you will: 
 - Run a hyperparameter sweep on  a training script
 - Compare the results of the runs in the MLFLOW UI
 - Choose the best run and register it as a model
 - Deploy the model to the REST API 
 - Build a container image suitable for the deployment to the cloud platform

In [ ]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature


: 

In [ ]:
# Load the data

data=pd.read_csv(
   "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv ",
    sep=";",
)
data.head()

In [3]:
## Split the data into training,validation and st sets 

train, test = train_test_split(data, test_size=0.2, random_state=42)

# Define the model



In [4]:
train_x = train.drop(["quality"], axis=1).values
train_y = train["quality"].values.ravel()

## Test Dataset
test_x = test.drop(["quality"], axis=1).values
test_y = test["quality"].values.ravel()

## Split the training data into training and validation sets
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)
signature = infer_signature(train_x,train_y)

# Define the search space



In [ ]:
### Ann Model

from numpy import shape


def train_model(prams,epochs,train_x,train_y,vali_x,valid_y,test_x,test_y):

    ##Define model architechture
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)

    model = keras.Sequential(
        [
            keras.Input([train_x,shape[1]]),
            keras.layers.normalization(mean = mean, varience = var),
            keras.layers.Dense(64, activation = 'relu'),
            keras.layers.Dense(1)
        ]
    )


    ##Compile the model
    model.compile(optimizer = keras.optimizers.SGD(
    learning_rate=params["lr"], momentum = params['momentum']
    ),
                 loss = "mean_squared_error",
                 metrics = [keras.metrics.RootMeanSquaredError()])